# To-Do:

- InputStandardize vs Normalize

- check hyperparam/prior definitions (scaled space vs raw space)

- keep in mind modality of NaN results in emittance

- Try different number of steps along measurement dimension

- inrease dimensionality of tuning space

- fix legend location (only plot on 1 heatmap)

# In this notebook, we fit a gpytorch GP to a simple emittance model with 1 tuning parameter. We use the GP to evaluate the Expected Information Gain toward the result of a grid-scan minimization routine.

In [1]:
import torch
from emitutils import toy_beam_size_squared_nd, fit_gp_model_emittance
from utils import unif_random_sample_domain
from matplotlib import pyplot as plt
from algorithms import GridMinimizeEmittance, GradBasedMinimizeEmittance
from acquisition import ExpectedInformationGain
from botorch.optim import optimize_acqf
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy

# Settings

In [2]:
# domain = torch.tensor([[-2,2], [-65,35]]).double() #the acquisition domain, must have shape = (ndim, 2)
domain = torch.tensor([[-3,1], [-40,60]]).double() #the acquisition domain, must have shape = (ndim, 2)
ndim = domain.shape[0]                               #where domain[i,0] and domain[i,1] represent
                                                        #the lower and upper bounds of the ith input dimension
                                                        #(these same bounds will be applied to the sampled execution paths) 

    

    
n_samples = 10 #number of posterior samples on which to evaluate execution paths
n_steps_tuning_params = 51 #number of steps per dimension in the posterior sample grid scans 
n_steps_measurement_param = 51
squared = True #whether or not to minimize the "emittance squared" (which can be negative according to the model)





random_acq = False
n_trials = 5
n_iter = 20
n_obs_init = 30 #number of random observations on which to initialize model


In [3]:
target_func = toy_beam_size_squared_nd

x_obs = unif_random_sample_domain(n_samples = n_obs_init, domain = domain)
y_obs = target_func(x_obs) 

model = fit_gp_model_emittance(x_obs, y_obs*1.e6)
algo = GradBasedMinimizeEmittance(domain = domain, 
               n_samples = n_samples, 
                n_iter = 100,
                n_steps_measurement_param = n_steps_measurement_param)

In [4]:
algo.get_exe_paths(model)

C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
C:\Users\Dylan\Anaconda3\envs\BOTORCH\lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(


tensor(1029.0723, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([[-2.8212],
        [-2.9633],
        [-0.1747],
        [-2.5251],
        [-1.3289],
        [ 0.1751],
        [ 0.9765],
        [ 0.0751],
        [-0.9592],
        [-1.2363]], dtype=torch.float64, grad_fn=<SliceBackward0>)
tensor(-4731.1314, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([[-2.8125],
        [-2.9533],
        [-0.1670],
        [-2.5177],
        [-1.3363],
        [ 0.1675],
        [ 0.9829],
        [ 0.0678],
        [-0.9667],
        [-1.2289]], dtype=torch.float64, grad_fn=<SliceBackward0>)
tensor(-4870.5483, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([[-2.8036],
        [-2.9435],
        [-0.1729],
        [-2.5155],
        [-1.3448],
        [ 0.1594],
        [ 0.9784],
        [ 0.0597],
        [-0.9729],
        [-1.2217]], dtype=torch.float64, grad_fn=<SliceBackward0>)
tensor(-7011.2928, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([[-2.8091],
        [-2.

KeyboardInterrupt: 

# Initialize